### **Uploading Kaggle Credentials**
This cell opens a file picker so we can upload the **kaggle.json** file. Colab needs this file to authenticate with Kaggle and access datasets.

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kaanhewa","key":"e8ced1639f7df8536282070139efdb1a"}'}

### **Setting Up Kaggle Configuration**
Creates the hidden **.kaggle** folder, places the kaggle.json file inside it, and adjusts permissions so only we can read it. This is required by the Kaggle CLI.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### **Installing the Kaggle Library**
Installs the official Kaggle package so the notebook can download datasets using Kaggle commands.

In [ ]:
!pip install kaggle

### **Downloading the Movie Plots Dataset**
Uses the Kaggle API to download the Wikipedia Movie Plots dataset as a ZIP file.

In [ ]:
!kaggle datasets download jrobischon/wikipedia-movie-plots

Dataset URL: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots
License(s): CC-BY-SA-4.0
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.07GB/s]


### **Extracting the Dataset**
Unzips all downloaded **.zip** files to make the CSV file available for reading.

In [ ]:
!unzip *.zip

Archive:  wikipedia-movie-plots.zip
  inflating: wiki_movie_plots_deduped.csv  


### **Loading the Dataset**
Reads the **wiki_movie_plots_deduped.csv** file into a pandas DataFrame and shows the first few rows.

In [ ]:
import pandas as pd

df = pd.read_csv("wiki_movie_plots_deduped.csv")
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


### **Checking Data Overview**
Shows column names, data types, missing values, and memory usage.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


### **Sampling & Selecting Columns**
Takes only the **Title** and **Plot** columns, then randomly selects 500 movies for analysis.

In [ ]:
subset = df[["Title", "Plot"]].sample(500, random_state=42)
subset

,Title,Plot
5337,The Day the Earth Stood Still,"When a flying saucer lands in Washington, D.C...."
9809,The Burning,"One night at Camp Blackfoot, several campers p..."
24075,Nobel Chor,"The first Asian Nobel Laureate, Rabindranath T..."
19057,Trent's Last Case,A major international financier is found dead ...
24991,Aafat,Inspector Amar and Inspector Chhaya are after ...
...,...,...
7324,Because They're Young,A crusading high school teacher tries to help ...
25974,Arjun Pandit,Arjun Pandit (Sunny Deol) is madly in love wit...
8370,Shalako,"In 1880 in New Mexico, frontier adventurer Bos..."
12082,Striking Distance,"Thomas Hardy, a Pittsburgh Police homicide det..."


### **Creating a Text Chunking Function**
Defines a function that splits long movie plots into smaller chunks of roughly 300 words.

In [ ]:
import pandas as pd

def chunk_text(text, max_words=300):

    if not isinstance(text, str):
        text = str(text)

    words = text.split()
    chunks = []

    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i + max_words])
        chunks.append(chunk)

    return chunks

### **Generating Chunks for All Movies**
Uses the chunking function to split each movie plot into pages. Builds a structured table with:

*   Movie title
*   Chunk number
*   Chunk text







In [ ]:
max_words = 300
rows = []

for _, row in subset.iterrows():
    title = row["Title"]
    plot = row["Plot"]

    plot_chunks = chunk_text(plot, max_words=max_words)

    for idx, ch in enumerate(plot_chunks):
        rows.append({
            "Title": title,
            "chunk_id": idx,
            "Plot_chunk": ch
        })

chunked_df = pd.DataFrame(rows)


### **Previewing the Chunked Dataset**
Displays the first few rows of the new chunked dataset.

In [ ]:
chunked_df.head()

,Title,chunk_id,Plot_chunk
0,The Day the Earth Stood Still,0,"When a flying saucer lands in Washington, D.C...."
1,The Day the Earth Stood Still,1,greatest living person is; Bobby suggests Prof...
2,The Day the Earth Stood Still,2,"the spaceship, then leaves to retrieve Klaatu'..."
3,The Burning,0,"One night at Camp Blackfoot, several campers p..."
4,The Burning,1,"Todd, Michelle, and the kids all go on a canoe..."


### **Installing ChromaDB and Gemini SDK**
Installs:


*   **ChromaDB** for vector storage
*   **Google Generative AI SDK** for calling Gemini

In [ ]:
!pip install chromadb google-generativeai

### **Creating a Vector Database & Adding Text Chunks**


1.   Initializes ChromaDB.
2.   Sets up an embedding model **(all-MiniLM-L6-v2)**.
3.   Creates a new collection named **movie_plots**.
4.   Converts each chunk to an embedding and stores:
  *   Text
  *   Movie title
  *   Chunk ID






In [ ]:
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

client = chromadb.Client(Settings(anonymized_telemetry=False))

embedding_fn = SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection = client.create_collection(
    name="movie_plots",
    embedding_function=embedding_fn
)

documents = chunked_df["Plot_chunk"].tolist()
metadatas = [
    {
        "Title": row["Title"],
        "chunk_id": int(row["chunk_id"])
    }
    for _, row in chunked_df.iterrows()
]
ids = [f"{i}" for i in range(len(documents))]

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas
)

len(ids)


941

### **Configuring the Gemini API Key**
Sets up the Gemini API key so the notebook can make calls to Google’s AI model.

In [ ]:
import google.generativeai as genai

genai.configure(api_key="")

### **Initializing Gemini Model**
Loads the **Gemini-2.5-Flash** model, which will generate answers later.

In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash")

### RAG Query Function
1. Retrieves top relevant plot chunks from ChromaDB.
2. Builds a structured context block.
3. Asks Gemini to answer using only the retrieved context.
4. Parses the model’s JSON response.
5. Returns
* The final answer
* The supporting contexts
* The model’s reasoning

In [ ]:
import json

def answer_query(query: str, top_k: int = 5):
    # 1) Retrieve from Chroma
    result = collection.query(query_texts=[query], n_results=top_k)
    docs = result["documents"][0]
    metas = result["metadatas"][0]

    # Build context block
    context_blocks = []
    for doc, meta in zip(docs, metas):
        context_blocks.append(
            f"Title: {meta['Title']}\n"
            f"Chunk ID: {meta['chunk_id']}\n"
            f"Plot snippet: {doc}"
        )
    context = "\n\n---\n\n".join(context_blocks)

    # 2) Prompt Gemini (2.5 Flash or your selected SKU)
    prompt = f"""
You are a movie expert answering questions using ONLY the context below.

Context:
{context}

User question:
{query}

Instructions:
- Use only the information in the context.
- If the answer is not clearly supported by the context, say you are not sure.
- Respond in valid JSON ONLY with this exact structure:
{{
  "answer": "<short natural language answer in 1–2 sentences>",
  "reasoning": "<short explanation (1–2 sentences) of how you used the context>"
}}
- Do NOT wrap JSON in markdown.
- Do NOT include code fences.
- Do NOT add any extra text outside the JSON.
"""

    response = model.generate_content(prompt)
    raw_text = (response.text or "").strip()

    # 3) Attempt strict JSON parsing
    try:
        model_json = json.loads(raw_text)
    except json.JSONDecodeError:
        # Fallback: treat model output as the answer
        model_json = {
            "answer": raw_text,
            "reasoning": "Model did not return strict JSON; raw output used as answer."
        }

    # 4) Final consolidated payload
    return {
        "answer": model_json.get("answer", "").strip(),
        "contexts": docs,
        "reasoning": model_json.get("reasoning", "").strip(),
    }


### **Running a Sample Query**
Asks the system:\
**“Which movie features a historical figure?”**



In [ ]:
query = "Which movie features an Historical figure?"
result_json = answer_query(query, top_k=5)

print(json.dumps(result_json, indent=2))

{
  "answer": "The movie \"Chanakya Chandragupta\" features historical figures such as the 3rd century BC Maurya Emperor Chandragupta, his advisor Chanakya, and Alexander The Great.",
  "contexts": [
    "The film is a historical story, based on life history the 3rd century BC Maurya Emperor Chandragupta (N. T. Rama Rao) & his advisor / Guru Chanakya (Akkineni Nageswara Rao). The story goes on how both of them destroyed Nanda Dynasty, defeated Alexander The Great (Sivaji Ganesan) and established Maurya Empire.",
    "The film tells a story with movie-making as the backdrop. Vijayaraghavan appears as the title character of Josettan, who is a senior film producer. Anoop Menon comes up as Saajan Malyath, the still photographer of the film who is accidentally selected to become the hero of the new film in production.",
    "The film tells the story of a revolutionary who, years later, realizes that his efforts have been wasted and witnesses the ill-fate of his co-rebels and leads a seclude